In [1]:
# import Python packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# import dataset
all_data = pd.read_csv('../input/nyc-housing-data-20032019/data.csv')

# split all_data into training and testing datasets for linear regression
training_data, testing_data = train_test_split(all_data, test_size = 0.2, random_state = 25)

/kaggle/input/nyc-housing-data-20032019/data.csv


In [2]:
# Step 1: understand the dataset

# shape of the training dataset and teh testing dataset
training_data #1280161 rows × 10 columns
testing_data #320041 rows × 10 columns

# understand the nature of teh training data
training_data.info()
training_data.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1280161 entries, 519590 to 29828
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   BOROUGH                  1280161 non-null  int64  
 1   NEIGHBORHOOD             1280161 non-null  object 
 2   BUILDING CLASS CATEGORY  1280161 non-null  object 
 3   ADDRESS                  1280161 non-null  object 
 4   ZIP CODE                 1280148 non-null  float64
 5   LAND SQUARE FEET         1266410 non-null  float64
 6   GROSS SQUARE FEET        1266411 non-null  float64
 7   YEAR BUILT               1275345 non-null  float64
 8   SALE PRICE               1280161 non-null  int64  
 9   SALE DATE                1280161 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 107.4+ MB


,BOROUGH,ZIP CODE,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,1.280161e+06,1.280148e+06,1.266410e+06,1.266411e+06,1.275345e+06,1.280161e+06
mean,2.915336e+00,1.080989e+04,4.175749e+03,4.401854e+03,1.727672e+03,8.572525e+05
std,1.310385e+00,7.635331e+02,4.720753e+05,5.237566e+04,6.181824e+02,1.173265e+07
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,1.030100e+04,0.000000e+00,0.000000e+00,1.916000e+03,0.000000e+00
50%,3.000000e+00,1.120800e+04,1.845000e+03,1.248000e+03,1.931000e+03,3.050000e+05
75%,4.000000e+00,1.135700e+04,2.800000e+03,2.352000e+03,1.964000e+03,6.215000e+05
max,5.000000e+00,3.380300e+04,2.147557e+08,1.496215e+07,2.020000e+03,4.111112e+09


In [3]:
# According to training_data.info(), the following columns have null values:
# Land square feet has 13751 null values
# Gross square feet has 13750 null values
# Year built has 4816 null values

# According to training_data.describe(), the following columns have values equal to 0 which they should not have:
# Land square feet
# Gross square feet
# Year built
# Sale price

# The following variables may need feature engineering:
# NEIGHBORHOOD
# BUILDING CLASS CATEGORY
# ADDRESS
# ZIP CODE

training_data.describe().columns
df_num = training_data[['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT']]
df_cat = training_data[['BOROUGH', 'ZIP CODE']]


